This is a 'clean code' version of the [draw_map](./draw_map.ipnb) notebook. The aim is to draw a map showing the number of inhabitants (2014 data) per one public transport (bus, tube or DLR) stop in each of the London wards, which could help answer the question how to improve London's public transport.

In [1]:
import xmltodict
from descartes import PolygonPatch
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize
from pysal.esda.mapclassify import Natural_Breaks as nb
from shapely.geometry import Point, MultiPoint, Polygon, MultiPolygon 
from shapely.prepared import prep
import pyproj
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
from __future__ import unicode_literals
import seaborn as sns

%matplotlib inline

from pylab import rcParams
rcParams['figure.figsize'] = (18,18)

We will be using several data sets in the analysis:

1. The map of the London wards (obrained from [here](http://data.london.gov.uk/dataset/statistical-gis-boundary-files-london)). To transform it into the WGS 84 reference system, we transofrm it: `$ ogr2ogr -t_srs EPSG:4326 London_Ward_ESPG4326.shp London_Ward_CityMerged.shp`.

2. The list of London bus stops (from Transport for London (TfL) [website](https://api-portal.tfl.gov.uk/docs)).

3. The list of London tube + DLR stops (also on the TfL website).

4. The estimate data about the inhabitants per ward ([here](http://data.london.gov.uk/dataset/office-national-statistics-ons-population-estimates-borough)).

First, we read in and prepare the data.

In [ ]:
# bus stops
busStops = pd.read_csv('data/bus-stops.csv')

# remove the virtual stops that are only for testing
busStops = busStops[busStops['Virtual_Bus_Stop'] == 0]

# the locations for the bus stops are in awkward units, need to transform them
def proj_transform(df):
    # Transforms from British Ordnance survey coords into lon and lat
    # credit: Alberto Negron 
    # http://altons.github.io/python/2012/12/01/converting-northing-and-easting-to-latitude-and-longitude/
    bng = pyproj.Proj(init='epsg:27700')
    wgs84 = pyproj.Proj(init='epsg:4326')
    lon,lat = pyproj.transform(bng,wgs84,list(df['Location_Easting']), list(df['Location_Northing']))
    
    df['lat'] = lat
    df['lon'] = lon
    return df

busStops = proj_transform(busStops)

In [ ]:
# tube + DLR stops
with open('data/stations.kml','r') as fd:
    doc = xmltodict.parse(fd.read())

tubeNames = []
tubeLats = []
tubeLons = []
    
for pMark in doc['kml']['Document']['Placemark']:
    #the actual data are hidden in 'Placemark's, all of which are OrderedDicts
    tubeNames.append(dict(pMark)['name'])
    #the Point key in pMark contains another OrderedDict with the coord string
    c = [float(num) for num in dict(dict(pMark)['Point'])['coordinates'].split(',')]
    tubeLons.append(c[0])
    tubeLats.append(c[1])

tubeStops = pd.DataFrame({'names': tubeNames, 'lon': tubeLons, 'lat': tubeLats})
